In [1]:
import os 
import glob #used to search for files in dir that match specific pattern 
import nibabel as nib 
from nibabel.processing import conform
import re #regular expression to match digit 
import numpy as np

In [ ]:
one = np.load('/data2/MRI_PET_DATA/graph/NACC/Morph/mri_atlas/morph/roi/NACC000511/001.npy')
print(one)

In [ ]:
#check if in mni space
#must be aligned in mni space or q/sform amtrix must match with the binary masks 
file = nib.load('/data2/MRI_PET_DATA/graph/ADNI/mri_atlas/wneuromorphometrics_0173_mri.nii')
print(file.header)

In [2]:
#os.path.join() concatenates path components with one / separator following each non empty part 
dir = "/home/mlauber"
file = 'fft_automate.ipynb'

new_path = os.path.join(dir, file)
print(new_path) #/home/mlauber/fft_automate.ipynb

#f strings 
#mri_file_path = os.path.join(mri_dir, f"{patient_id}_mri.nii") 
#tells it that whatever patient_id is equal to for current iteration, join the mri_dir /data2/MRI_PET_DATA/graph/ADNI/ADNI_MRI_nii_recentered_cat12_cox_noqc with the patient id so that its /data2/MRI_PET_DATA/graph/ADNI/ADNI_MRI_nii_recentered_cat12_cox_noqc/6652_mri.nii


/home/mlauber/fft_automate.ipynb


In [3]:
#ADNI 
# #location of the bin masks for each segment 
# mask_dir="/data2/MRI_PET_DATA/graph/ADNI/mri_atlas/roi"  #subfolders wneuromorphometrics_####_mri.nii each with rois 142.nii

# #location of the og mri aligned in the mni space 
# mri_dir="/data2/MRI_PET_DATA/graph/ADNI/ADNI_MRI_nii_recentered_cat12_cox_noqc"  #files ####_mri.nii 

#NACC 
#location of the bin masks for each segment 
mask_dir="/data2/MRI_PET_DATA/graph/NACC/Morph/mri_atlas/morph/roi"  #subfolders wneuromorphometrics_####_mri.nii each with rois 142.nii

#location of the og mri aligned in the mni space 
mri_dir="/data2/MRI_PET_DATA/graph/NACC/ADNI_MRI_NACC_recenter_cox_test"  #files ####_mri.nii 


In [7]:
#testing .gz files loading 

file = nib.load("/data2/MRI_PET_DATA/graph/ADNI/mri_atlas/roi/wneuromorphometrics_0173/masked_133.nii.gz").get_fdata()
print(file.shape)

(160, 240, 256)


In [11]:
#NACC signal masks 


#for each directory that matches the pattern NACC[0-9][0-9][0-9][0-9][0-9], os.path.join joins "/data2/MRI_PET_DATA/graph/NACC/Morph/mri_atlas/morph/roi"  with NACC[0-9][0-9][0-9][0-9][0-9] and iterates over each directory 
for dir_path in sorted(glob.glob(os.path.join(mask_dir,"NACC[0-9][2-9][0-9][0-9][0-9][0-9]"))): 
    #assign patient ID based on the original mri instead since its having trouble 
    print(dir_path)
    #dir_path = /data2/MRI_PET_DATA/graph/ADNI/mri_atlas/roi/wneuromorphometrics_6652

    #its going into dir, looping through and resaving each mask as masked_nii.nii 

    #the nacc bin mask folders are NACC###### (6 digit id)
    patient_id = dir_path[-6:]
    print(f"Converting masks for {patient_id}......")
    # print(patient_id) #6652 so it's extracting all the ids 

#just finished 019767
#pick up from one after it 

    #instead of searching for all .nii files, only search for '[0-9][0-9][0-9].nii' files so it doesn't keep looping over same folder 
    for bin_mask_file_path in sorted(glob.glob(os.path.join(dir_path, '[0-9][0-9][0-9].nii.gz'))): #specifies bin mask file paths /data2/MRI_PET_DATA/graph/NACC/Morph/mri_atlas/morph/roi

        mri_file_path = os.path.join(mri_dir, f"NACC{patient_id}_mri.nii") #change this to nacc filepath /data2/MRI_PET_DATA/graph/NACC/ADNI_MRI_NACC_recenter_cox_test
        #naming convention of mri images NACC######_mri.nii

        # print(mri_file_path) 
        # print(bin_mask_file_path)
    
    #check if mri exists for the patient id 
    #not finding mri file 
        if os.path.isfile(mri_file_path):
            # print(f"Converting mask file: {bin_mask_file_path}") 
            # print(f"Using MRI file: {mri_file_path}") #ex #mri path is /data2/MRI_PET_DATA/graph/ADNI/ADNI_MRI_nii_recentered_cat12_cox_noqc/6652_mri.nii

            #get 3 digit region number from bin mask file name 
            #os.path.basename returns the basename of the file path whic is filename itself with extension 001.nii
            #os.path.splittext()[0] #splittext splits file base name into tuple of the file name iteself and the extension (001, .nii) so [0] tells it to access the first element of the tuple which is 001 
            reg_num = (os.path.splitext(os.path.join(bin_mask_file_path)))[0][-7:-4:] #extract last 3 elements 
            # print(reg_num)
            sig_mask_file_path = os.path.join(dir_path, f"masked_{reg_num}.nii")
            # print(f"converting region {reg_num}")
            #reg_num var is set as .nii because its the last 3 before the extension 
            # print(sig_mask_file_path) #/data2/MRI_PET_DATA/graph/ADNI/mri_atlas/roi/wneuromorphometrics_6652/014.nii

            mri_img = nib.load(mri_file_path)
            # print(mri_img.shape)
            mask_img = nib.load(bin_mask_file_path)
            # print(mask_img.shape)

            #resize 
            resize_mask_img = nib.processing.conform(mask_img, out_shape = mri_img.shape)
            # print(resize_mask_img.shape)

            mri_array = mri_img.get_fdata()
            # print(patient_id,mri_array.shape)
            resize_mask_array = resize_mask_img.get_fdata()
            # print(patient_id,resize_mask_array.shape)

            #multiply roi by original mri to get signal mask
            sig_mask = mri_array * resize_mask_array  #files are different shapes so need to add in line to convert the images 
            # print(sig_mask.shape)

            #save output as nifti file
            nib.save(nib.Nifti2Image(sig_mask,nib.load(mri_file_path).affine), sig_mask_file_path)
            # print(f"Mask file saved for {bin_mask_file_path[-12:]}")
            # print(f"Mask files saved for {bin_mask_file_path}") #Mask file saved for 0173/001.nii
        else:
            print(f"Error: MRI file not found for: {patient_id}")

    print(f"Mask files saved for {patient_id}")
    continue

/data2/MRI_PET_DATA/graph/NACC/Morph/mri_atlas/morph/roi/NACC000511
Converting masks for 000511......
Mask files saved for 000511
/data2/MRI_PET_DATA/graph/NACC/Morph/mri_atlas/morph/roi/NACC000531
Converting masks for 000531......
Mask files saved for 000531
/data2/MRI_PET_DATA/graph/NACC/Morph/mri_atlas/morph/roi/NACC003196
Converting masks for 003196......
Mask files saved for 003196
/data2/MRI_PET_DATA/graph/NACC/Morph/mri_atlas/morph/roi/NACC006009
Converting masks for 006009......
Mask files saved for 006009
/data2/MRI_PET_DATA/graph/NACC/Morph/mri_atlas/morph/roi/NACC006420
Converting masks for 006420......
Mask files saved for 006420
/data2/MRI_PET_DATA/graph/NACC/Morph/mri_atlas/morph/roi/NACC007377
Converting masks for 007377......
Mask files saved for 007377
/data2/MRI_PET_DATA/graph/NACC/Morph/mri_atlas/morph/roi/NACC008987
Converting masks for 008987......
Mask files saved for 008987
/data2/MRI_PET_DATA/graph/NACC/Morph/mri_atlas/morph/roi/NACC011143
Converting masks for 0

In [4]:

#ADNI signal masks 
#4743 is missinh signal masks for 58-114, so I need to go back and reprocess those and then process and compress them 
for dir_path in sorted(glob.glob(os.path.join(mask_dir,"wneuromorphometrics_[4][7][4][3]"))): #for each directory that matches the pattern wneuromorphometrics_[0-9][0-9][0-9][0-9, os.path.join joins "/data2/MRI_PET_DATA/graph/ADNI/mri_atlas/roi"  with /wneuromorphometrics_#### and iterates over each directory 
    #assign patient ID based on the original mri instead since its having trouble 
    # print(dir_path)
    #dir_path = /data2/MRI_PET_DATA/graph/ADNI/mri_atlas/roi/wneuromorphometrics_6652

    #its going into dir, looping through and resaving each mask as masked_nii.nii 
    #fix it because it used to 
    #without .gz though it just loops throiugh without converting bc nothing matches pattern 
    #2363 start 

    patient_id = dir_path[-4:]
    print(f"Converting masks for {patient_id}......")
    # print(patient_id) #6652 so it's extracting all the ids 

    #instead of searching for all .nii files, only search for '[0-9][0-9][0-9].nii' files so it doesn't keep looping over same folder 
    for bin_mask_file_path in sorted(glob.glob(os.path.join(dir_path, '[0-9][0-9][0-9].nii.gz'))): #specifies mask file paths are /data2/MRI_PET_DATA/graph/ADNI/mri_atlas/roi/wneuromorphometrics_6652/*.nii 
        mri_file_path = os.path.join(mri_dir, f"{patient_id}_mri.nii") #mri path is /data2/MRI_PET_DATA/graph/ADNI/ADNI_MRI_nii_recentered_cat12_cox_noqc/6652_mri.nii
        # print(mri_file_path) 
        # print(bin_mask_file_path)
    
    #check if mri exists for the patient id 
        if os.path.isfile(mri_file_path):
            # print(f"Converting mask file: {bin_mask_file_path}") #ex /data2/MRI_PET_DATA/graph/ADNI/mri_atlas/roi/wneuromorphometrics_6652/044.nii 
            # print(f"Using MRI file: {mri_file_path}") #ex #mri path is /data2/MRI_PET_DATA/graph/ADNI/ADNI_MRI_nii_recentered_cat12_cox_noqc/6652_mri.nii

            #for 2363 might have to go back and delete the duplicate .nii files where there are already .gz files 
            #they're saved as .nii so have to go back and unzip them 

            #get 3 digit region number from bin mask file name 
            #os.path.basename returns the basename of the file path whic is filename itself with extension 001.nii
            #os.path.splittext()[0] #splittext splits file base name into tuple of the file name iteself and the extension (001, .nii) so [0] tells it to access the first element of the tuple which is 001 
            reg_num = (os.path.splitext(os.path.join(bin_mask_file_path)))[0][-7:-4:] #extract last 3 elements 
            # print(reg_num)
            sig_mask_file_path = os.path.join(dir_path, f"masked_{reg_num}.nii")
            # print(f"converting region {reg_num}")
            #reg_num var is set as .nii because its the last 3 before the extension 
            # print(sig_mask_file_path) #/data2/MRI_PET_DATA/graph/ADNI/mri_atlas/roi/wneuromorphometrics_6652/014.nii

            mri_img = nib.load(mri_file_path)
            # print(mri_img.shape)
            mask_img = nib.load(bin_mask_file_path)
            # print(mask_img.shape)

            #resize 
            resize_mask_img = nib.processing.conform(mask_img, out_shape = mri_img.shape)
            # print(resize_mask_img.shape)

            mri_array = mri_img.get_fdata()
            # print(patient_id,mri_array.shape)
            resize_mask_array = resize_mask_img.get_fdata()
            # print(patient_id,resize_mask_array.shape)

            #multiply roi by original mri to get signal mask
            sig_mask = mri_array * resize_mask_array  #files are different shapes so need to add in line to convert the images 
            # print(sig_mask.shape)

            #save output as nifti file
            nib.save(nib.Nifti2Image(sig_mask,nib.load(mri_file_path).affine), sig_mask_file_path)
            # print(f"Mask file saved for {bin_mask_file_path[-12:]}")
            # print(f"Mask files saved for {bin_mask_file_path}") #Mask file saved for 0173/001.nii
        else:
            print(f"Error: MRI file not found for: {patient_id}")

    print(f"Mask files saved for {patient_id}")
    continue

Converting masks for 4743......
Mask files saved for 4743
